In [24]:
import re
import time
from selenium import webdriver
import jieba
import wordcloud
from lxml import etree
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
import traceback
from selenium.webdriver.support.ui import WebDriverWait
from webdriver_manager.chrome import ChromeDriverManager
import easygui as g #图形界面

option = webdriver.ChromeOptions()
option.add_argument('headless')
head={}
head['User-Agent']='Mozilla/5.0 (Windows NT 6.3; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/80.0.3987.163 Safari/537.36'

def get_number(name):
    driver = webdriver.Chrome(ChromeDriverManager().install())
    url='https://search.douban.com/movie/subject_search?search_text='+name+'&cat=1002'
    driver.get(url)
    link1 = re.findall(r'<a href="https://movie.douban.com/subject/(\d*)/"', driver.page_source)
    return link1[0]
    
def get_data(url):
    data = ''
    driver = webdriver.Chrome(ChromeDriverManager().install(),chrome_options=option) #打开chrome浏览器
    wait = WebDriverWait(driver, 10)  #显示等待，等待是否有界面，10秒后无则报错
    driver.get(url) #获取源代码
    for i in range(10): #最多查看10页
        print("正在爬取第{}页".format(i+1))
        html = etree.HTML(driver.page_source, etree.HTMLParser()) 
        for a in range(20): #每页最多有20条短评
            data += str(html.xpath('//*[@id="comments"]/div['+str(a+1)+']/div[2]/p/span/text()')) #Xpath 
        #第二种表达：data += str(html.xpath('//*[@id="comments"]/div['+str(a+1)+']/div[2]/p/span/text()') for i in range(0, 21))
        try: #第一页的后一页selector与后面24页的不同
            if i == 0:
                confirm_btn = wait.until(EC.element_to_be_clickable((By.CSS_SELECTOR, '#paginator > a'))) 
                #通过seletector进行定位点击的位置——后一页
            else:
                confirm_btn = wait.until(EC.element_to_be_clickable((By.CSS_SELECTOR, '#paginator > a.next')))
                #通过seletector进行定位点击的位置——后一页
        except: #打印被调用方的错误行及错误信息
            '''
            print ('=== STEP ERROR INFO START')
            traceback.print_exc()
            print ('=== STEP ERROR INFO END')
            '''
            return data
        else:
            confirm_btn.click() #点击这个confirm_btn按钮，进行翻页
        time.sleep(5) # 推迟执行5秒
    return data

def write_data(name,data): #转成txt文件
    f=open(name+'影评.txt','w',encoding='utf-8')
    f.write(data)
    f.close()
    
def word_cloud(name):
    txt = open(name+"影评.txt",'r',encoding='utf-8').read()
    words = jieba.lcut(txt)
    yingming = jieba.lcut(name)
    f = open(r'stoplist.txt','r',encoding='utf-8')
    stop_data = f.readlines()
    stopwords = []
    for i in stop_data:
        a= i.replace('\n', '').replace('\r', '')
        stopwords.append(a)
    stopwords = stopwords + yingming
    counts = {} #词频字典

    excludes ={
    '一个','书友','故事','自己','这个','这是','没有','不是','成为','可以','之中','群号','什么',
    '知道','本书','开始','不能','大家','来到','一切','......','验证','欢迎','但是 '
    '踏上','完本','这里','一群','你们','别人','已经','一起','之后','一次','从此','这种'
    '发现','就是','时候','不想','只是','还是','一只','...','简介','完结','如何'
    ,'后来','怎么','那个','每天','本文','我们','一天','只有','还有','部分','那么','但是'
    ,'为了','这样','他们','不要','真的','以为','不过','作者','最后','感觉','觉得','因为'
    ,'作为','理解','一本','虽然','而且','这么','这种','这本','反正','有点','非常','为什么'
    ,'读完','看到','也许','甚至','很多','多么','一直','简直','有些','应该','之前','东西'
    ,'一种','其实','对于','起来','看过','里面','比较','下去','出现','无法','真是','完全'
    ,'确实','可能','如果','一般','只能','之间','然后','从来','代替','总是','全部','因为'
    ,'所以','足够','不断','一些','大概','充满','一定','而是','当时','仿佛','如此','之外'
    ,'那些'
    }

    for word in words: #算词频
        if len(word) == 1:
            continue
        else:
            rword = word
        counts[rword] = counts.get(rword,0) + 1

    for word in  stopwords: #删停用词
        if word in counts :
            del counts[word]
    for word in excludes:  #删人工停用词
        if word in counts :
            del counts[word]
    items = list(counts.items())
    items.sort(key=lambda x:x[1],reverse=True) #排序
    counts = ''
    for i in range(100):
        word = items[i][0]
        count = items[i][1]
        for i in range(count):
            counts += word+' '
        print('{0:<10}{1:<5}'.format(word,count))
    w = wordcloud.WordCloud(font_path = 'msyh.ttc',width=800, height=400,background_color='white',collocations=False) #生成词云图
    w.generate(counts)
    w.to_file(name+'.png') #生成文件

def main():
    try:
        name = g.enterbox(msg="请输入影名",title="评论词云生成器")
        url = 'https://movie.douban.com/subject/'+get_number(name)+'/comments?status=P' #指定的电影评论网址 p=1表示为第一页
        data = get_data(url)
        write_data(name,data)
        word_cloud(name)
        g.buttonbox(msg='',image=name+'.png' ,title='评论词云生成器', choices=(''))
        return 1
    except:
        return 0

if __name__=="__main__":
    main()



[WDM] - Cache is valid for [06/06/2020]
[WDM] - Looking for [chromedriver 83.0.4103.39 win32] driver in cache 
[WDM] - Driver found in cache [C:\Users\果壳恩\.wdm\drivers\chromedriver\83.0.4103.39\win32\chromedriver.exe]


[WDM] - Cache is valid for [06/06/2020]
[WDM] - Looking for [chromedriver 83.0.4103.39 win32] driver in cache 
[WDM] - Driver found in cache [C:\Users\果壳恩\.wdm\drivers\chromedriver\83.0.4103.39\win32\chromedriver.exe]


D:\Anaconda3\lib\site-packages\ipykernel_launcher.py:28: DeprecationWarning: use options instead of chrome_options


正在爬取第1页
正在爬取第2页
正在爬取第3页
正在爬取第4页
正在爬取第5页
正在爬取第6页
正在爬取第7页
正在爬取第8页
正在爬取第9页
正在爬取第10页
中国        124  
科幻        109  
电影        96   
特效        51   
科幻片       42   
人类        41   
科幻电影      40   
国产        36   
设定        33   
人物        33   
宇宙        31   
这部        30   
煽情        30   
台词        29   
好莱坞       28   
希望        28   
鼓励        27   
吴京        25   
剧情        23   
大片        22   
一部        22   
观众        22   
原著        22   
值得        21   
尴尬        20   
情感        20   
叙事        19   
影片        19   
刘慈欣       18   
导演        18   
太空        18   
三星        18   
拯救        18   
类型        17   
木星        17   
元年        17   
宏大        16   
情节        16   
灾难片       16   
意义        15   
片子        15   
战狼        15   
穿越        14   
技术        14   
特别        14   
剪辑        14   
文化        14   
角色        14   
救援        14   
郭帆        14   
演员        14   
画面        14   
价值观       13   
星际        13   
地方        13   
世界        13   
人性        13   
不错     